In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### settings

In [2]:
chrome_driver = "C:/Users/seungsu/Desktop/gentlegraph/chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("headless")
chrome_options.add_argument("--window-size=1400,1080")

chrome_prefs = {}
chrome_options.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2,'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2,'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2}

driver = webdriver.Chrome(chrome_driver, options=chrome_options)

### 전체 재생목록 가져오기

In [3]:
import time

driver.get("https://www.youtube.com/playlist?list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R")
driver.execute_script('window.scrollTo(0, 20000);')
driver.implicitly_wait(2)
playlist = driver.find_elements_by_css_selector("ytd-playlist-video-renderer")

In [4]:
total_contents = []
for content in playlist:
    index = content.find_element_by_css_selector("#index").text
    video_info = content.find_element_by_css_selector("a#video-title")
    channel_info = content.find_element_by_css_selector("ytd-channel-name#channel-name").find_element_by_css_selector("a")
    url = video_info.get_attribute("href")
    title = video_info.text
    creator = channel_info.text
    creator_channel = channel_info.get_attribute("href")
    
    data = {
        "rank" : index,
        "content_url" : url,
        "content_title" : title,
        "creator" : creator,
        "creator_url" : creator_channel
    }
    
    total_contents.append(data)

In [5]:
total_contents

[{'rank': '1',
  'content_url': 'https://www.youtube.com/watch?v=v7bnOxV4jAc&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1',
  'content_title': '[MV] IU(아이유)_LILAC(라일락)',
  'creator': '1theK (원더케이)',
  'creator_url': 'https://www.youtube.com/c/1theK'},
 {'rank': '2',
  'content_url': 'https://www.youtube.com/watch?v=J_sPEzOsixE&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=2',
  'content_title': '(ENG) 나 엑소 좋아했네? 솔직히 여기 나온 노래 모르는 사람 대한민국 국민 아님(그만큼 명곡이 많다는 뜻) / [문명특급 EP.177]',
  'creator': '문명특급 - MMTG',
  'creator_url': 'https://www.youtube.com/c/Oopssofastexpress'},
 {'rank': '3',
  'content_url': 'https://www.youtube.com/watch?v=6xz1bay-6dQ&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=3',
  'content_title': '[#하이라이트#]드디어 만남 성사! 진짜 BTS와 BTS 여고생의 만남! 살아계세요?#유퀴즈온더블럭 | YOU QUIZ ON THE BLOCK EP.99',
  'creator': '유 퀴즈 온 더 튜브',
  'creator_url': 'https://www.youtube.com/channel/UC920m3pMPH45qztdhppZhwA'},
 {'rank': '4',
  'content_url': 'https://www.youtube.com/watch?v=9yneHTModes&list=P

### 세부 사항 크롤링

In [6]:
content = total_contents[0]
content

{'rank': '1',
 'content_url': 'https://www.youtube.com/watch?v=v7bnOxV4jAc&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1',
 'content_title': '[MV] IU(아이유)_LILAC(라일락)',
 'creator': '1theK (원더케이)',
 'creator_url': 'https://www.youtube.com/c/1theK'}

In [9]:
driver.get(content["content_url"])
tag = driver.find_element_by_css_selector("#movie_player > div.html5-video-container > video")
tag.click()

button = driver.find_element_by_css_selector("#more")
button.click()

driver.execute_script('window.scrollTo(0, 1000);')
driver.implicitly_wait(2)

In [10]:
views = driver.find_element_by_css_selector("#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer").text
date = driver.find_element_by_css_selector("#date > yt-formatted-string").text
good = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a").find_element_by_css_selector("#text").text
bad = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a").find_element_by_css_selector("#text").text
comments = driver.find_element_by_css_selector("#count > yt-formatted-string > span:nth-child(2)").text

In [12]:
description = driver.find_element_by_css_selector("#description")
hashlist = description.find_elements_by_css_selector("a")
texts = description.find_elements_by_css_selector("span")

hashtags = [tag.text for tag in hashlist if tag.text not in ("", " ")]        
descriptions = [span.text for span in texts if span.text not in (" ", "")]

In [11]:
views, date, good, bad, comments

('조회수 6,929,992회', '2021. 3. 25.', '76만', '7.7천', '44,504')

In [13]:
hashtags

['https://www.youtube.com/1theKOriginals',
 '#NEWRELEASE',
 '#MV',
 '#1theK',
 '#원더케이',
 'https://www.youtube.com/1theK',
 'http://www.facebook.com/1theK',
 'https://twitter.com/1theK',
 'https://goo.gl/otRpZc',
 'https://vt.tiktok.com/2mSMBS']

In [14]:
descriptions

['[MV] IU(아이유)_LILAC(라일락)\n\n\nK-POP Wonderland, 1theK\n\nK-POP의 모든 즐거움을 1theK(원더케이)에서 만나보세요! :)\nWelcome to the official YouTube channel of K-POP Wonderland, 1theK\n\n""1theK Originals"" Subscribe 👉 ',
 '[Notice] 1theK YouTube is also an official channel for the MV, and music shows will count the views from this channel too.\n[공지] 1theK YouTube는 MV를 유통하는 공식 채널로, 1theK에 업로드된 MV 조회수 또한 음악방송 순위에 반영됩니다.',
 '▶1theK YT : ',
 '▶1theK FB  : ',
 '▶1theK TW : ',
 '▶1theK Kakao : ',
 '▶1theK TikTok : ',
 '"']

In [1]:
import re

def str_to_int(string : str):
    value = 0
    if '만' in string:
        value = float(re.findall('\d+\.*\d*', string)[0])
        value *= 10000
    elif '천' in string:
        value = float(re.findall('\d+\.*\d*', string)[0])
        value *= 1000
    else:
        tmp = re.findall('\d+\.*\d*', string)
        if len(tmp) == 0:
            value = 0
        else:
            value = float(tmp[0])
        
    return int(value)

In [14]:
def crawl_detail_content(, data : dict) -> dict:
    
    if data == None:
        data = {}
    
    # go to content detail page
    driver.get(content["content_url"])
    tag = driver.find_element_by_css_selector("#movie_player > div.html5-video-container > video")
    tag.click()
    
    try:
        button = driver.find_element_by_css_selector("paper-button#more")
        button.click()
    except:
        pass

    driver.execute_script('window.scrollTo(0, 1500);')
    driver.implicitly_wait(10)
    
    # get detail information of content
    views = driver.find_element_by_css_selector("#count > ytd-video-view-count-renderer > span.view-count.style-scope.ytd-video-view-count-renderer").text.replace(",", "")
    views = re.findall('\d+', views)[0]
    date = driver.find_element_by_css_selector("#date > yt-formatted-string").text.replace(".", "").split()
    date = f"{date[0]}-{date[1]}-{date[2]}"
    good = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(1) > a").find_element_by_css_selector("#text").text
    
    bad = driver.find_element_by_css_selector("#top-level-buttons > ytd-toggle-button-renderer:nth-child(2) > a").find_element_by_css_selector("#text").text
    try:
        comments = driver.find_element_by_css_selector("#count > yt-formatted-string > span:nth-child(2)").text
        comments = int(comments.replace(",", ""))
    except:
        comments = -1
    time.sleep(2)
    # get content description and hashtags
    description = driver.find_element_by_css_selector("#description")
    hashlist = description.find_elements_by_css_selector("a")
    texts = description.find_elements_by_css_selector("span")
    hashtags = [tag.text for tag in hashlist if tag.text not in ("", " ") and tag.text.startswith("#")]        
    descriptions = [span.text for span in texts if span.text not in (" ", "")]
    
    # append detail data
    data.update(
        {
            "views" : views,
            "date" : date,
            "good" : str_to_int(good),
            "bad" : str_to_int(bad),
            "comment" : comments,
            "hashtags" : hashtags,
            "description" : descriptions
        }
    )
    
    print(data)
    
    return data
    

### 전체 크롤링

In [19]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13490 sha256=aed6c067c3ec92b3decbf7adc28b2604a975379951fe9859b71b423c095d08f4
  Stored in directory: C:\Users\seungsu\AppData\Local\pip\Cache\wheels\5e\63\09\d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
from fake_useragent import UserAgent

In [21]:
# init chrome driver
chrome_driver = "C:/Users/seungsu/Desktop/gentlegraph/chromedriver_win32/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument("headless")
chrome_options.add_argument("--window-size=1100,2000")
userAgent = UserAgent().random
chrome_options.add_argument(f'user-agent={userAgent}')

prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}} 
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_driver, options=chrome_options)


# get playlists
driver.get("https://www.youtube.com/playlist?list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R")
driver.execute_script('window.scrollTo(0, 20000);')
driver.implicitly_wait(5)
time.sleep(3)
playlist = driver.find_elements_by_css_selector("ytd-playlist-video-renderer.style-scope.ytd-playlist-video-list-renderer")

print(f"total data : {len(playlist)}")

# get brief info
total_contents = []
for content in tqdm(playlist):
    index = content.find_element_by_css_selector("#index").text
    video_info = content.find_element_by_css_selector("a#video-title")
    channel_info = content.find_element_by_css_selector("ytd-channel-name#channel-name").find_element_by_css_selector("a")
    url = video_info.get_attribute("href")
    title = video_info.text
    creator = channel_info.text
    creator_channel = channel_info.get_attribute("href")
    
    data = {
        "rank" : index,
        "content_url" : url,
        "content_title" : title,
        "creator" : creator,
        "creator_url" : creator_channel
    }
    
    total_contents.append(data)

# get detail info
for content in tqdm(total_contents):
    crawl_detail_content(content["content_url"], content)

total data : 200




  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

  0%|▍                                                                                 | 1/200 [00:00<00:21,  9.37it/s]

  1%|▊                                                                                 | 2/200 [00:00<00:21,  9.25it/s]

  2%|█▏                                                                                | 3/200 [00:00<00:21,  9.33it/s]

  2%|█▋                                                                                | 4/200 [00:00<00:20,  9.35it/s]

  2%|██                                                                                | 5/200 [00:00<00:21,  9.26it/s]

  3%|██▍                                                                               | 6/200 [00:00<00:20,  9.35it/s]

  4%|██▊                                                                               | 7/200 [00:00<00:20,  9.36it/s]

  4%|███▎                     

 68%|██████████████████████████████████████████████████████                          | 135/200 [00:16<00:08,  7.49it/s]

 68%|██████████████████████████████████████████████████████▍                         | 136/200 [00:16<00:08,  7.67it/s]

 68%|██████████████████████████████████████████████████████▊                         | 137/200 [00:17<00:08,  7.47it/s]

 69%|███████████████████████████████████████████████████████▏                        | 138/200 [00:17<00:08,  7.54it/s]

 70%|███████████████████████████████████████████████████████▌                        | 139/200 [00:17<00:07,  7.78it/s]

 70%|████████████████████████████████████████████████████████                        | 140/200 [00:17<00:08,  7.36it/s]

 70%|████████████████████████████████████████████████████████▍                       | 141/200 [00:17<00:07,  7.41it/s]

 71%|████████████████████████████████████████████████████████▊                       | 142/200 [00:17<00:07,  7.63it/s]

 72%|███████████████████████████

{'rank': '1', 'content_url': 'https://www.youtube.com/watch?v=v7bnOxV4jAc&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=1', 'content_title': '[MV] IU(아이유)_LILAC(라일락)', 'creator': '1theK (원더케이)', 'creator_url': 'https://www.youtube.com/c/1theK', 'views': '7368819', 'date': '2021-3-25', 'good': 780000, 'bad': 7900, 'comment': -1, 'hashtags': ['#NEWRELEASE', '#MV', '#1theK', '#원더케이'], 'description': ['[MV] IU(아이유)_LILAC(라일락)\n\n\nK-POP Wonderland, 1theK\n\nK-POP의 모든 즐거움을 1theK(원더케이)에서 만나보세요! :)\nWelcome to the official YouTube channel of K-POP Wonderland, 1theK\n\n""1theK Originals"" Subscribe 👉 ', '[Notice] 1theK YouTube is also an official channel for the MV, and music shows will count the views from this channel too.\n[공지] 1theK YouTube는 MV를 유통하는 공식 채널로, 1theK에 업로드된 MV 조회수 또한 음악방송 순위에 반영됩니다.', '▶1theK YT : ', '▶1theK FB  : ', '▶1theK TW : ', '▶1theK Kakao : ', '▶1theK TikTok : ', '"']}




  0%|▍                                                                               | 1/200 [00:19<1:03:09, 19.04s/it]

...

{'rank': '23', 'content_url': 'https://www.youtube.com/watch?v=LV8Ezog59cI&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=23', 'content_title': 'Brave Girls, Rollin’(브레이브걸스, 롤린) [THE SHOW 210323]', 'creator': 'The K-POP', 'creator_url': 'https://www.youtube.com/c/TheKPOP', 'views': '798096', 'date': '2021-3-23', 'good': 28000, 'bad': 252, 'comment': -1, 'hashtags': ['#브레이브걸스', '#Brave_Girls', '#Rollin'], 'description': [', 롤린', '’\n\n채널에 가입하여 혜택을 누려보세요.', 'The K-POP \n\nAll about K-POP in Korea!\nOfficial  K-POP YouTube channel of SBS Medianet.\nPlease don’t forget to click Subscribe.\nEnjoy your YouTube life with THE K-POP.']}




 12%|█████████▎                                                                       | 23/200 [08:46<55:50, 18.93s/it]

{'rank': '24', 'content_url': 'https://www.youtube.com/watch?v=8lb2rhoeieA&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=24', 'content_title': '붙여서 먹는 미래식량으로만 3일 버티기', 'creator': '공대생 변승주 DS', 'creator_url': 'https://www.youtube.com/c/%EA%B3%B5%EB%8C%80%EC%83%9D%EC%A3%BC%EC%A3%BC', 'views': '359160', 'date': '2021-3-25', 'good': 6400, 'bad': 198, 'comment': -1, 'hashtags': ['#대리흑우', '#붙히는영양소', '#리뷰'], 'description': ['문의는 gds@videovillage.co.kr\nhjsbjh2@gmail.com 으로 부탁드려요!\n\nTrack : 귀여운 BGM 모음\nMusic by 브금대통령\nMusic provided by 브금대통령\nWatch : ']}




 12%|█████████▋                                                                       | 24/200 [09:03<54:25, 18.55s/it]

KeyboardInterrupt: 

In [17]:
content = total_contents[24]
crawl_detail_content(content["content_url"], content)

{'rank': '25', 'content_url': 'https://www.youtube.com/watch?v=yIwooGxe1dg&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=25', 'content_title': '당신이 쿠지라이식 간짬뽕을 절대 알 수 없는 이유(마지막 소름 주의)', 'creator': '쿠왕coo king', 'creator_url': 'https://www.youtube.com/channel/UCR-qpXziP5pIqNHFKL-w12g', 'views': '1046553', 'date': '2021-3-22', 'good': 15000, 'bad': 1000, 'comment': -1, 'hashtags': ['#shorts'], 'description': ['당신이 쿠지라이식 간짬뽕을 알 수 없는 이유에 대해서 진지하게 알려드립니다. ', '문의 : cooking7564@gmail.com\n\n🎵 브금대통령\n🎵Track :  ']}


{'rank': '25',
 'content_url': 'https://www.youtube.com/watch?v=yIwooGxe1dg&list=PLmtapKaZsgZt3g_uAPJbsMWdkVsznn_2R&index=25',
 'content_title': '당신이 쿠지라이식 간짬뽕을 절대 알 수 없는 이유(마지막 소름 주의)',
 'creator': '쿠왕coo king',
 'creator_url': 'https://www.youtube.com/channel/UCR-qpXziP5pIqNHFKL-w12g',
 'views': '1046553',
 'date': '2021-3-22',
 'good': 15000,
 'bad': 1000,
 'comment': -1,
 'hashtags': ['#shorts'],
 'description': ['당신이 쿠지라이식 간짬뽕을 알 수 없는 이유에 대해서 진지하게 알려드립니다. ',
  '문의 : cooking7564@gmail.com\n\n🎵 브금대통령\n🎵Track :  ']}